models:
llama 3 - meta-llama/Llama-3.1-8B
gemini - google/gemma-7b
chatglm - THUDM/chatglm-6b
CPM - TsinghuaAI/CPM-Generate


start filenames:
1_meta-llama_Llama-3.1-8B_results_noun
1_google_gemma-7b_results_noun
1_THUDM_chatglm-6b_results_noun
1_TsinghuaAI_CPM-Generate_results_noun
1_meta-llama_Llama-3.1-8B_results_adjective
1_google_gemma-7b_results_adjective
1_THUDM_chatglm-6b_results_adjective
1_TsinghuaAI_CPM-Generate_results_adjective

end filenames:
2_meta-llama_Llama-3.1-8B_results_noun
2_google_gemma-7b_results_noun
2_THUDM_chatglm-6b_results_noun
2_TsinghuaAI_CPM-Generate_results_noun
2_meta-llama_Llama-3.1-8B_results_adjective
2_google_gemma-7b_results_adjective
2_THUDM_chatglm-6b_results_adjective
2_TsinghuaAI_CPM-Generate_results_adjective

In [ ]:
import pandas as pd
import itertools

In [ ]:
filename = f'../results/1_meta-llama_Meta-Llama-3-8B_results_noun.csv'
df = pd.read_csv(filename, on_bad_lines='warn')

df

Sentence-level apx per individual ⬇️   

In [ ]:
for i in range(1,4):
    cluster_summary = []

    overall_mean = df[f'perplexity_{i}'].mean()

    for group in df['Gender group'].unique():
        cluster_data = df[(df['Gender group'] == group)]
        
        # Get the average perplexity for the current group
        cluster_gender = cluster_data['Gender group'].iloc[0]
        average_perplexity = round(cluster_data[f'perplexity_{i}'].mean(), 2)
        prop_perplexity = round(cluster_data[f'perplexity_{i}'].mean() / overall_mean, 3)  # average of the group / overall

        cluster_summary.append({'Group': cluster_gender, 
        'Average Perplexity': average_perplexity, 'Proportional Perplexity': prop_perplexity})

    # Create a DataFrame from the cluster summary list
    prop_df = pd.DataFrame(cluster_summary)

    # Sort the DataFrame by perplexity in ascending order
    prop_df = prop_df.sort_values(by='Average Perplexity')

    # Merge the DataFrames based on 'Group'
    df = pd.merge(df, prop_df, on='Group', how='left')

    # Calculate Adjusted Perplexity
    df[f'apx_{i}'] = df[f'perplexity_{i}'] / df['Proportional Perplexity']
    df = df.drop(['Average Perplexity', 'Proportional Perplexity'], axis=1)

Group-level average per descriptor ⬇️

In [ ]:
def get_summary_df(df):
    cluster_summary = []
    
    for group in df['Gender group'].unique():
        for descriptor in df['descriptor'].unique():
            group_df = df[(df['Gender group'] == group)
                        & (df['descriptor'] == descriptor)]

            # Check if any rows exist for the current combination and group_id
            if not group_df.empty:
                # Get the ethnicity, gender, and average perplexity for the current cluster
                average_apx_1 = round(group_df['apx_1'].mean(), 2)
                average_apx_2 = round(group_df['apx_2'].mean(), 2)
                average_apx_3 = round(group_df['apx_3'].mean(), 2)

                cluster_summary.append({'Group': group, 'apx_1': average_apx_1, 
                                        'apx_2': average_apx_2, 'apx_3': average_apx_3,
                                        'descriptor': descriptor, 'axis': group_df['axis'].iloc[0],
                                        'gender_nouns': group_df['Gender-specific nouns'].iloc[0],
                                        'des_type': group_df['des_type'].iloc[0]})

    # Create a DataFrame from the cluster summary list
    summary_df = pd.DataFrame(cluster_summary)

    # Sort the DataFrame by descriptor in ascending order
    summary_df = summary_df.sort_values(by='descriptor')

    return(summary_df)

In [ ]:
summary_df = get_summary_df(df)
adj_name="llama_Llama-3.1-8B"
summary_df.to_csv(f'../results/2_{adj_name}_results_noun.csv')